In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as psf

In [3]:
spark = SparkSession.builder.appName("Spark Train Line Quiz").getOrCreate()

In [39]:
cities_df = spark.read.csv('./cities.csv', header=True); cities_df.show(1)

+---+--------+------------------+----------+--------+--------+-------------+
| id|    name|            coords|start_year|url_name| country|country_state|
+---+--------+------------------+----------+--------+--------+-------------+
|  5|Aberdeen|POINT(-2.15 57.15)|      2017|aberdeen|Scotland|         null|
+---+--------+------------------+----------+--------+--------+-------------+
only showing top 1 row



In [40]:
lines_df = spark.read.csv('./lines.csv', header=True)
tracks_df = spark.read.csv('./tracks.csv', header=True)

In [41]:
lines_df.show(1)

+---+-------+-------+----------+-------+---------+-----------------+
| id|city_id|   name|  url_name|  color|system_id|transport_mode_id|
+---+-------+-------+----------+-------+---------+-----------------+
| 43|      4|Línea 2|43-linea-2|#ffbe2e|      267|                4|
+---+-------+-------+----------+-------+---------+-----------------+
only showing top 1 row



In [42]:
tracks_df.show(1)

+----+--------------------+----------+-------+-------+------+-------+
|  id|            geometry|buildstart|opening|closure|length|city_id|
+----+--------------------+----------+-------+-------+------+-------+
|1911|LINESTRING(19.081...|         0|      0| 999999|  6719|     29|
+----+--------------------+----------+-------+-------+------+-------+
only showing top 1 row



In [46]:
lines_df = lines_df.withColumnRenamed('name', 'city_name').select("city_id", "city_name")

In [47]:
lines_df.show()

+-------+--------------------+
|city_id|           city_name|
+-------+--------------------+
|      4|             Línea 2|
|     34|Línea 3 Metro de ...|
|    126|             Línea 1|
|     74|          La navette|
|     66|  Línea 1 (Tramo 1A)|
|     91|  Tokaido Shinkansen|
|      1|                   A|
|    261|             Línea 1|
|    110|                  14|
|      4|             Línea 4|
|      4|            Línea 4A|
|      4|             Línea 5|
|      4|             Línea 6|
|      4|       MetroTren Nos|
|    110|                  12|
|     79|                  M1|
|      4|Ramal San Felipe ...|
|     79|                  M5|
|     21|    Troncal C - Suba|
|    118|                 22A|
+-------+--------------------+
only showing top 20 rows



In [48]:
left_df = cities_df.join(lines_df, cities_df.id == lines_df.city_id, "inner"); left_df.show()

+---+----------------+--------------------+----------+----------------+-------------+-------------+-------+------------------+
| id|            name|              coords|start_year|        url_name|      country|country_state|city_id|         city_name|
+---+----------------+--------------------+----------+----------------+-------------+-------------+-------+------------------+
|211|          Ottawa|POINT(-75.7166666...|      2000|          ottawa|       Canada|         Ont.|    211|   2 Trillium Line|
|147|         Chicago|POINT(-87.6166666...|      1892|         chicago|United States|         Ill.|    147|Garfield Park Line|
|147|         Chicago|POINT(-87.6166666...|      1892|         chicago|United States|         Ill.|    147|        Brown Line|
|147|         Chicago|POINT(-87.6166666...|      1892|         chicago|United States|         Ill.|    147|       Yellow Line|
|147|         Chicago|POINT(-87.6166666...|      1892|         chicago|United States|         Ill.|    147|    

In [49]:
tracks_df.select(psf.max("closure")).distinct().show()

+------------+
|max(closure)|
+------------+
|   999999999|
+------------+



In [50]:
filtered_track_df = tracks_df.filter("closure like '999999%'"); filtered_track_df.show()

+----+--------------------+----------+-------+-------+------+-------+
|  id|            geometry|buildstart|opening|closure|length|city_id|
+----+--------------------+----------+-------+-------+------+-------+
|1911|LINESTRING(19.081...|         0|      0| 999999|  6719|     29|
|2563|LINESTRING(16.415...|         0|      0| 999999|   199|    118|
|2557|LINESTRING(16.416...|         0|      0| 999999|   925|    118|
|2558|LINESTRING(16.416...|         0|      0| 999999|   881|    118|
|2564|LINESTRING(16.415...|         0|      0| 999999|   213|    118|
|2565|LINESTRING(16.412...|         0|      0| 999999|   321|    118|
|2566|LINESTRING(16.412...|         0|      0| 999999|   320|    118|
|2567|LINESTRING(16.394...|         0|      0| 999999|  1555|    118|
|3434|LINESTRING(139.38...|      1926|   1926| 999999|  3534|    114|
|3433|LINESTRING(139.40...|      1921|   1921| 999999|  4966|    114|
|8350|LINESTRING(-0.470...|         0|      0| 999999|    17|    256|
|6469|LINESTRING(-1.

In [25]:
filtered_track_df.select('id').dropDuplicates().count()

8451

In [51]:
joined_df = left_df.join(filtered_track_df, left_df.city_id==filtered_track_df.city_id, how='inner'); joined_df.show(5)

+---+-------+--------------------+----------+--------+-------------+-------------+-------+------------------+----+--------------------+----------+-------+-------+------+-------+
| id|   name|              coords|start_year|url_name|      country|country_state|city_id|         city_name|  id|            geometry|buildstart|opening|closure|length|city_id|
+---+-------+--------------------+----------+--------+-------------+-------------+-------+------------------+----+--------------------+----------+-------+-------+------+-------+
|147|Chicago|POINT(-87.6166666...|      1892| chicago|United States|         Ill.|    147|Garfield Park Line|4549|LINESTRING(-87.63...|         0|   1897| 999999|  1588|    147|
|147|Chicago|POINT(-87.6166666...|      1892| chicago|United States|         Ill.|    147|Garfield Park Line|1993|LINESTRING(-87.63...|         0|   1900| 999999|  7321|    147|
|147|Chicago|POINT(-87.6166666...|      1892| chicago|United States|         Ill.|    147|Garfield Park Line|1

In [54]:
joined_df.select('city_name').distinct().count()

1035

In [ ]:
    # TODO Q1: how do you know which track is still operating?
    tracks_df.select(psf.max("closure")).distinct().show()

    # TODO Q2: filter on only the operating tracks
    filtered_df = tracks_df.filter("closure like '999999%' ")

    joined_df = left_df.join(filtered_df, left_df.city_id == filtered_df.city_id, "inner")

    # TODO Q1 and Q2 answers
    joined_df.select("city_name").distinct().count()
    joined_df.select("city_name").distinct().show(40)